# Подключение гугл диска и загрузка библиотек

In [ ]:
%%capture
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
# drive.flush_and_unmount()

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle as pkl
import math
import torch
import os
import pandas as pd
import requests
import tensorflow as tf
import random
import glob
import gc
from ultralytics import YOLO
from tensorflow.keras.preprocessing.image import load_img, array_to_img, ImageDataGenerator, img_to_array
from glob import glob
from PIL import Image
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

# Подготовка папок временной сессии Colab

In [ ]:
%cd /content/
!mkdir images
!mkdir labels
!mkdir train
%cd /content/train
!mkdir images
!mkdir labels
%cd /content
!mkdir valid
%cd /content/valid
!mkdir images
!mkdir labels
%cd /content/

/content
/content/train
/content
/content/valid
/content


In [ ]:
# уже аугментированные данные
!unzip /content/drive/MyDrive/финал/augmentated/images.zip
!unzip /content/drive/MyDrive/финал/augmentated/labels.zip

Archive:  /content/drive/MyDrive/финал/augmentated/images2q.zip
  inflating: images2/0_1_1_0.jpg     
  inflating: images2/0_1_1_1.jpg     
  inflating: images2/0_1_1_2.jpg     
  inflating: images2/0_1_1_3.jpg     
  inflating: images2/0_1_1_4.jpg     
  inflating: images2/0_1_1_5.jpg     
  inflating: images2/0_1_2_0.jpg     
  inflating: images2/0_1_2_1.jpg     
  inflating: images2/0_1_2_2.jpg     
  inflating: images2/0_1_2_3.jpg     
  inflating: images2/0_1_2_4.jpg     
  inflating: images2/0_1_2_5.jpg     
  inflating: images2/0_1_3_0.jpg     
  inflating: images2/0_1_3_1.jpg     
  inflating: images2/0_1_3_2.jpg     
  inflating: images2/0_1_3_3.jpg     
  inflating: images2/0_1_3_4.jpg     
  inflating: images2/0_1_3_5.jpg     
  inflating: images2/0_1_4_0.jpg     
  inflating: images2/0_1_4_1.jpg     
  inflating: images2/0_1_4_2.jpg     
  inflating: images2/0_1_4_3.jpg     
  inflating: images2/0_1_4_4.jpg     
  inflating: images2/0_1_4_5.jpg     
  inflating: images2/0_1

# Тренировка с помощью кросс-валидации

In [ ]:
# # перемещение в основные папки
# paths = [pth[:-4] for pth in os.listdir('/content/train/images')]
# for pth in paths:
#     os.replace(f"/content/train/images/{pth}.jpg", f'/content/images/{pth}.jpg')
# paths = [pth[:-4] for pth in os.listdir('/content/train/labels')]
# for pth in paths:
#     os.replace(f"/content/train/labels/{pth}.txt", f'/content/labels/{pth}.txt')

# paths = [pth[:-4] for pth in os.listdir('/content/valid/images')]
# for pth in paths:
#     os.replace(f"/content/valid/images/{pth}.jpg", f'/content/images/{pth}.jpg')
# paths = [pth[:-4] for pth in os.listdir('/content/valid/labels')]
# for pth in paths:
#     os.replace(f"/content/valid/labels/{pth}.txt", f'/content/labels/{pth}.txt')

# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# %cd /content/drive/MyDrive/
# class_0 = [pth[:-4] for pth in os.listdir('/content/images') if pth[0] == '0']
# class_1 = [pth[:-4] for pth in os.listdir('/content/images') if pth[0] == '1']
# class_2 = [pth[:-4] for pth in os.listdir('/content/images') if pth[0] == '2']
# control_0 = random.sample(class_0, k=int(len(class_0) / 6))
# control_1 = random.sample(class_1, k=int(len(class_1) / 6))
# control_2 = random.sample(class_2, k=int(len(class_2) / 6))
# # print(len(class_0), len(class_1), len(class_2), len(control_0), len(control_1), len(control_2))
# class_0 = set(class_0) - set(control_0)
# class_1 = set(class_1) - set(control_1)
# class_2 = set(class_2) - set(control_2)
# with open('/content/drive/MyDrive/control0.txt', 'wt') as f:
#     f.write(' '.join([str(i) for i in control_0]))
# with open('/content/drive/MyDrive/control1.txt', 'wt') as f:
#     f.write(' '.join([str(i) for i in control_1]))
# with open('/content/drive/MyDrive/control2.txt', 'wt') as f:
#     f.write(' '.join([str(i) for i in control_2]))
# paths = list(class_0) + list(class_1) + list(class_2)
# # paths = [pth[:-4] for pth in os.listdir('/content/images')]
# df = pd.DataFrame(data=paths, columns=['PATH']).sample(frac = 1)
# kf = KFold(n_splits=5, shuffle=True, random_state=40)
# for train_index, val_index in kf.split(df):
#     val = [path['PATH'].split('/')[-1] for ndx, path in df.iloc[list(val_index)].iterrows()]
#     train = [path['PATH'].split('/')[-1] for ndx, path in df.iloc[list(train_index)].iterrows()]
#     # print(len(val), len(train))
#     for pth in val:
#         os.replace(f'/content/images/{pth}.jpg', f"/content/valid/images/{pth}.jpg")
#         os.replace(f'/content/labels/{pth}.txt', f"/content/valid/labels/{pth}.txt")
#     for pth in train:
#         os.replace(f'/content/images/{pth}.jpg', f"/content/train/images/{pth}.jpg")
#         os.replace(f'/content/labels/{pth}.txt', f"/content/train/labels/{pth}.txt")


#     model = YOLO('yolov8l.pt')  # load a pretrained model (recommended for training)

#     results = model.train(data='/content/data.yaml', epochs=50, imgsz=640, val=True, save_period=25)


#     for pth in val:
#         os.replace(f"/content/valid/images/{pth}.jpg", f'/content/images/{pth}.jpg')
#         os.replace(f"/content/valid/labels/{pth}.txt", f'/content/labels/{pth}.txt')
#     for pth in train:
#         os.replace(f"/content/train/images/{pth}.jpg", f'/content/images/{pth}.jpg')
#         os.replace(f"/content/train/labels/{pth}.txt", f'/content/labels/{pth}.txt')
#     gc.collect()
#     torch.cuda.empty_cache()

/content/drive/MyDrive


Ultralytics YOLOv8.0.200 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=25, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=Fal

KeyboardInterrupt: ignored

In [ ]:
%cd /content/drive/MyDrive/
class_0_prev = ['/content/images/' + '_'.join(pth[:-4].split('_')[:2]) for pth in os.listdir('/content/images') if pth[0] == '0']
class_1_prev = ['/content/images/' + '_'.join(pth[:-4].split('_')[:2]) for pth in os.listdir('/content/images') if pth[0] == '1']
class_2_prev = ['/content/images/' + '_'.join(pth[:-4].split('_')[:2]) for pth in os.listdir('/content/images') if pth[0] == '2']
print(sorted(list(np.array([[path for path in os.listdir('/content/images') if '_'.join(path.split('_')[:-2]).startswith(pth.split("/")[-1])] for pth in class_0_prev]).flat)))
class_0 = sorted(list(set(list(np.array([[path for path in os.listdir('/content/images') if '_'.join(path.split('_')[:-2]).startswith(pth.split("/")[-1])] for pth in class_0_prev]).flat))))
# print(sorted(list(set(['_'.join(path.split('_')[:-2]) for path in os.listdir('/content/images')]))))
# class_1 = ['/content/images/' + '_'.join(pth[:-4].split('_')[:2]) for pth in class_1_prev if pth[0] == '1']
# class_2 = ['/content/images/' + '_'.join(pth[:-4].split('_')[:2]) for pth in class_2_prev if pth[0] == '2']
print(sorted(class_0), sorted(class_0_prev), sep='\n')



# control_0 = random.sample(class_0, k=int(len(class_0) / 6))
# control_1 = random.sample(class_1, k=int(len(class_1) / 6))
# control_2 = random.sample(class_2, k=int(len(class_2) / 6))
# # print(len(class_0), len(class_1), len(class_2), len(control_0), len(control_1), len(control_2))
# class_0 = set(class_0) - set(control_0)
# class_1 = set(class_1) - set(control_1)
# class_2 = set(class_2) - set(control_2)
# with open('/content/drive/MyDrive/control0.txt', 'wt') as f:
#     f.write(' '.join([str(i) for i in control_0]))
# with open('/content/drive/MyDrive/control1.txt', 'wt') as f:
#     f.write(' '.join([str(i) for i in control_1]))
# with open('/content/drive/MyDrive/control2.txt', 'wt') as f:
#     f.write(' '.join([str(i) for i in control_2]))
# paths = list(class_0) + list(class_1) + list(class_2)
# # paths = [pth[:-4] for pth in os.listdir('/content/images')]
# df = pd.DataFrame(data=paths, columns=['PATH']).sample(frac = 1)
# kf = KFold(n_splits=5, shuffle=True, random_state=40)
# for train_index, val_index in kf.split(df):
#     val = [path['PATH'].split('/')[-1] for ndx, path in df.iloc[list(val_index)].iterrows()]
#     train = [path['PATH'].split('/')[-1] for ndx, path in df.iloc[list(train_index)].iterrows()]
#     # print(len(val), len(train))
#     for pth in val:
#         os.replace(f'/content/images/{pth}.jpg', f"/content/valid/images/{pth}.jpg")
#         os.replace(f'/content/labels/{pth}.txt', f"/content/valid/labels/{pth}.txt")
#     for pth in train:
#         os.replace(f'/content/images/{pth}.jpg', f"/content/train/images/{pth}.jpg")
#         os.replace(f'/content/labels/{pth}.txt', f"/content/train/labels/{pth}.txt")


#     model = YOLO('yolov8l.pt')  # load a pretrained model (recommended for training)

#     results = model.train(data='/content/data.yaml', epochs=50, imgsz=640, val=True, save_period=25, name='night19')


#     for pth in val:
#         os.replace(f"/content/valid/images/{pth}.jpg", f'/content/images/{pth}.jpg')
#         os.replace(f"/content/valid/labels/{pth}.txt", f'/content/labels/{pth}.txt')
#     for pth in train:
#         os.replace(f"/content/train/images/{pth}.jpg", f'/content/images/{pth}.jpg')
#         os.replace(f"/content/train/labels/{pth}.txt", f'/content/labels/{pth}.txt')
#     gc.collect()
#     torch.cuda.empty_cache()

/content/drive/MyDrive
[['0_10_3_2.jpg', '0_10_1_4.jpg', '0_10_1_5.jpg', '0_10_4_1.jpg'], ['0_10_3_2.jpg', '0_10_1_4.jpg', '0_10_1_5.jpg', '0_10_4_1.jpg'], ['0_10_3_2.jpg', '0_10_1_4.jpg', '0_10_1_5.jpg', '0_10_4_1.jpg'], ['0_10_3_2.jpg', '0_10_1_4.jpg', '0_10_1_5.jpg', '0_10_4_1.jpg'], ['0_11_2_1.jpg', '0_11_4_0.jpg', '0_11_1_0.jpg', '0_11_2_0.jpg', '0_11_2_5.jpg'], ['0_11_2_1.jpg', '0_11_4_0.jpg', '0_11_1_0.jpg', '0_11_2_0.jpg', '0_11_2_5.jpg'], ['0_11_2_1.jpg', '0_11_4_0.jpg', '0_11_1_0.jpg', '0_11_2_0.jpg', '0_11_2_5.jpg'], ['0_11_2_1.jpg', '0_11_4_0.jpg', '0_11_1_0.jpg', '0_11_2_0.jpg', '0_11_2_5.jpg'], ['0_11_2_1.jpg', '0_11_4_0.jpg', '0_11_1_0.jpg', '0_11_2_0.jpg', '0_11_2_5.jpg'], ['0_12_4_1.jpg', '0_12_1_1.jpg', '0_12_3_1.jpg', '0_12_3_4.jpg', '0_12_2_3.jpg'], ['0_12_4_1.jpg', '0_12_1_1.jpg', '0_12_3_1.jpg', '0_12_3_4.jpg', '0_12_2_3.jpg'], ['0_12_4_1.jpg', '0_12_1_1.jpg', '0_12_3_1.jpg', '0_12_3_4.jpg', '0_12_2_3.jpg'], ['0_12_4_1.jpg', '0_12_1_1.jpg', '0_12_3_1.jpg', '0_12_3

<ipython-input-62-2e7d002ebdca>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(sorted(list(np.array([[path for path in os.listdir('/content/images') if '_'.join(path.split('_')[:-2]).startswith(pth.split("/")[-1])] for pth in class_0_prev]).flat)))
<ipython-input-62-2e7d002ebdca>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  class_0 = sorted(list(set(list(np.array([[path for path in os.listdir('/content/images') if '_'.join(path.split('_')[:-2]).startswith(pth.split("/")[-1])] for pth in class_0_prev]).flat))))


TypeError: ignored

# Проверка и предсказание для теста соответственно

In [ ]:
def check(classes):
    if not len(classes):
        return 3
    return int(max(set(classes), key=classes.count))

In [ ]:
# # перемещение в основные папки
# paths = [pth[:-4] for pth in os.listdir('/content/train/images')]
# for pth in paths:
#     os.replace(f"/content/train/images/{pth}.jpg", f'/content/images/{pth}.jpg')
# paths = [pth[:-4] for pth in os.listdir('/content/train/labels')]
# for pth in paths:
#     os.replace(f"/content/train/labels/{pth}.txt", f'/content/labels/{pth}.txt')

# paths = [pth[:-4] for pth in os.listdir('/content/valid/images')]
# for pth in paths:
#     os.replace(f"/content/valid/images/{pth}.jpg", f'/content/images/{pth}.jpg')
# paths = [pth[:-4] for pth in os.listdir('/content/valid/labels')]
# for pth in paths:
#     os.replace(f"/content/valid/labels/{pth}.txt", f'/content/labels/{pth}.txt')

# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
model = YOLO('/content/drive/MyDrive/weights/best(3).pt')
with open('/content/drive/MyDrive/control0.txt', 'rt') as f:
    test0 = f.readline().split(' ')
with open('/content/drive/MyDrive/control1.txt', 'rt') as f:
    test1 = f.readline().split(' ')
with open('/content/drive/MyDrive/control2.txt', 'rt') as f:
    test2 = f.readline().split(' ')
tested0 = []
for img in test0:
    result = check(list(map(int, model(f'/content/images/{img}.jpg', device='0')[0].boxes.cls.tolist())))
    tested0.append(result)
tested1 = []
for img in test1:
    result = check(list(map(int, model(f'/content/images/{img}.jpg')[0].boxes.cls.tolist())))
    tested1.append(result)
tested2 = []
for img in test2:
    result = check(list(map(int, model(f'/content/images/{img}.jpg')[0].boxes.cls.tolist())))
    tested2.append(result)
print(f'f1-score of class number one: {f1_score(np.full((1, len(tested0)), 0)[0], tested0, average="weighted")}\n' +
      f'f1-score of class number two: {f1_score(np.full((1, len(tested1)), 1)[0], tested1, average="weighted")}\n' +
      f'f1-score of class number three: {f1_score(np.full((1, len(tested2)), 2)[0], tested2, average="weighted")}')
# confidence


image 1/1 /content/images/0_10_4_3.jpg: 640x640 1 Cirsium, 59.9ms
Speed: 6.5ms preprocess, 59.9ms inference, 41.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/images/0_16_9_0.jpg: 640x640 1 Cirsium, 1 Sonchus, 57.8ms
Speed: 1.9ms preprocess, 57.8ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/images/0_17_10_2.jpg: 640x640 1 Cirsium, 1 Sonchus, 50.4ms
Speed: 1.6ms preprocess, 50.4ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/images/0_15_5_5.jpg: 640x640 1 Cirsium, 50.2ms
Speed: 1.2ms preprocess, 50.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/images/0_17_8_2.jpg: 640x640 1 Cirsium, 50.3ms
Speed: 1.3ms preprocess, 50.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/images/0_1_2_0.jpg: 640x640 1 Cirsium, 34.5ms
Speed: 1.2ms preprocess, 34.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640

f1-score of class number one: 0.9949748743718593
f1-score of class number two: 0.9947089947089947
f1-score of class number three: 0.9333333333333332


In [ ]:
print(f'f1-score of class number one: {f1_score(np.full((1, len(tested0)), 0)[0], tested0, average="weighted")}\n' +
      f'f1-score of class number two: {f1_score(np.full((1, len(tested1)), 1)[0], tested1, average="weighted")}\n' +
      f'f1-score of class number three: {f1_score(np.full((1, len(tested2)), 2)[0], tested2, average="weighted")}')

f1-score of class number one: 0.9949748743718593
f1-score of class number two: 0.9947089947089947
f1-score of class number three: 0.9333333333333332
